# Clasificación de Tumores con MLP (PyTorch)

Este notebook implementa una red neuronal de perceptrón multicapa (MLP) utilizando PyTorch para predecir si un tumor es benigno o maligno.

## Paso 1: Cargar y explorar los datos
Se carga el dataset, se eliminan columnas irrelevantes y se convierten las variables categóricas a formato numérico.

In [1]:
import pandas as pd

# Cargar el dataset
data = pd.read_csv("../data/breast-cancer-wisconsin-data.csv")

# Eliminar la columna 'id' y convertir 'diagnosis' a valores binarios
data = data.drop(columns=['id'])
data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0})

# Mostrar información general del dataset
print(data.info())
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    int64  
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

## Paso 2: Preprocesamiento de los datos
Normalizamos las características y dividimos los datos en conjuntos de entrenamiento y prueba.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Dividir en características y etiqueta
X = data.drop(columns=['diagnosis'])
y = data['diagnosis']

# Normalizar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Verificar las formas de los conjuntos
print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

Conjunto de entrenamiento: (455, 30), (455,)
Conjunto de prueba: (114, 30), (114,)


## Paso 3: Implementar la red MLP con PyTorch
Diseñamos una red neuronal básica con PyTorch, utilizando una capa oculta con 32 neuronas.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# Modelo MLP
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.layer1 = nn.Linear(input_dim, 32)
        self.relu = nn.ReLU()
        self.output = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.sigmoid(self.output(x))
        return x

# Crear el modelo
input_dim = X_train.shape[1]
model = MLP(input_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Mostrar la arquitectura del modelo
print(model)

MLP(
  (layer1): Linear(in_features=30, out_features=32, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


## Paso 4: Entrenar el modelo
Entrenamos el modelo utilizando el conjunto de entrenamiento y monitorizamos el rendimiento.

In [4]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

# Entrenar el modelo
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor).squeeze()
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Época {epoch + 1}/{epochs}, Pérdida: {loss.item():.4f}")

Época 1/50, Pérdida: 0.6643
Época 2/50, Pérdida: 0.6520
Época 3/50, Pérdida: 0.6401
Época 4/50, Pérdida: 0.6284
Época 5/50, Pérdida: 0.6170
Época 6/50, Pérdida: 0.6058
Época 7/50, Pérdida: 0.5950
Época 8/50, Pérdida: 0.5844
Época 9/50, Pérdida: 0.5740
Época 10/50, Pérdida: 0.5639
Época 11/50, Pérdida: 0.5541
Época 12/50, Pérdida: 0.5445
Época 13/50, Pérdida: 0.5350
Época 14/50, Pérdida: 0.5258
Época 15/50, Pérdida: 0.5168
Época 16/50, Pérdida: 0.5080
Época 17/50, Pérdida: 0.4994
Época 18/50, Pérdida: 0.4909
Época 19/50, Pérdida: 0.4826
Época 20/50, Pérdida: 0.4745
Época 21/50, Pérdida: 0.4665
Época 22/50, Pérdida: 0.4586
Época 23/50, Pérdida: 0.4510
Época 24/50, Pérdida: 0.4434
Época 25/50, Pérdida: 0.4360
Época 26/50, Pérdida: 0.4287
Época 27/50, Pérdida: 0.4216
Época 28/50, Pérdida: 0.4146
Época 29/50, Pérdida: 0.4077
Época 30/50, Pérdida: 0.4009
Época 31/50, Pérdida: 0.3943
Época 32/50, Pérdida: 0.3878
Época 33/50, Pérdida: 0.3814
Época 34/50, Pérdida: 0.3751
Época 35/50, Pérdida: 0

## Paso 5: Evaluar el modelo
Evaluamos el modelo en el conjunto de prueba utilizando métricas como precisión, recall, F1-score y matriz de confusión.

In [5]:
from sklearn.metrics import classification_report, confusion_matrix

# Convertir conjunto de prueba a tensores
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Predecir en el conjunto de prueba
model.eval()
with torch.no_grad():
    y_pred = (model(X_test_tensor).squeeze() > 0.5).int().numpy()

# Generar métricas de clasificación
print(classification_report(y_test, y_pred))

# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(conf_matrix)

              precision    recall  f1-score   support

           0       0.95      0.99      0.97        71
           1       0.97      0.91      0.94        43

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114

Matriz de Confusión:
[[70  1]
 [ 4 39]]


## Conclusión
Analizamos el rendimiento del modelo y discutimos posibles mejoras para la red MLP.